In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from tqdm import tqdm
from time import time

In [5]:
cd ..

/home/jovyan/Ames-Iowa-Data


In [6]:
df = pd.read_csv('data/final_ames_df.csv')

In [7]:
final_ames_df = pd.DataFrame(df)

In [8]:
final_ames_df = final_ames_df.drop(['Unnamed: 0'], axis = 1)
target = final_ames_df['SalePrice']
features = final_ames_df.drop(['SalePrice'], axis = 1)

In [9]:
def time_function_call(function_call):
    start = time()
    result = function_call
    execution_time = time() - start
    return result, execution_time

def run_model(model, model_name, features, target):
    X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42)
    
    _, fit_time = time_function_call(model.fit(X_train, y_train))
    
    _, train_pred_time = time_function_call(
        model.predict(X_train))
    
    _, test_pred_time = time_function_call(
        model.predict(X_test)) 
  
    return {
        'model' : model,
        'model_name' : model_name,
        'train_score' : model.score(X_train, y_train),
        'test_score' : model.score(X_test, y_test),
        'fit_time' : fit_time,
        'train_pred_time': train_pred_time,
        'test_pred_time': test_pred_time
    }

model_scores = []

### Ridge Regression

Ridge performs a linear regression with an L1-regularization. 
The loss function is described using the Least Absolute Error (LAE).

$$ S = \sum_{i=1}^n |y_i - f(x_i)|$$


In [10]:
model_scores.append(run_model(Ridge(), 'ridge', features, target))

### Lasso Regression

Lasso performs a linear regression with an L2-regularization. The loss function is described using the Least Squares Error (LSE).
When running Lasso, the alpha is set at 0.1. 
$$ S = \sum_{i=1}^n (y_i - f(x_i))^2$$

In [11]:
model_scores.append(run_model(Lasso(alpha = 0.1), 'lasso', features, target))

### K Nearest Neighbors Regressor

The target is predicted by local interpolation of the targets associated of the nearest neighbors in the training set. The default number of neighbors = 5.

In [12]:
model_scores.append(run_model(KNeighborsRegressor(), 'knr', features, target))

### Decision Tree Regressor

The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features.

In [13]:
model_scores.append(run_model(DecisionTreeRegressor(), 'dtr', features, target))

### Support Vector Machine Regressor

The default kernel used in SVR is `rbf` or Radial Basis Function kernel.
The default penalty is set at `C=0.1`.

In [14]:
model_scores.append(run_model(SVR(), 'svr', features, target))

### Model Comparisons

In [15]:
model_scores = pd.DataFrame(model_scores)
cols = model_scores.columns.tolist()
cols = ['model_name','model', 'train_score', 'test_score', 'fit_time', 'train_pred_time', 'test_pred_time']
model_scores[cols]

,model_name,model,train_score,test_score,fit_time,train_pred_time,test_pred_time
0,ridge,"Ridge(alpha=1.0, copy_X=True, fit_intercept=Tr...",0.959360,0.868772,1.430511e-06,7.152557e-07,1.430511e-06
1,lasso,"Lasso(alpha=0.1, copy_X=True, fit_intercept=Tr...",0.816814,0.820449,9.536743e-07,7.152557e-07,7.152557e-07
2,knr,"KNeighborsRegressor(algorithm='auto', leaf_siz...",0.841907,0.742240,1.192093e-06,7.152557e-07,9.536743e-07
3,dtr,"DecisionTreeRegressor(criterion='mse', max_dep...",1.000000,0.666819,9.536743e-07,1.192093e-06,9.536743e-07
4,svr,"SVR(C=1.0, cache_size=200, coef0=0.0, degree=3...",0.934353,0.731889,1.192093e-06,1.430511e-06,1.430511e-06


Ridge and Lasso models "out of the box" show the highest $R^2$ test_scores with values of `0.868772` and `0.820449`.

Decision Tree Regressor had a train_score of `1.000` and the lowest of the test_scores `0.666819` which indicates that the model is overfit.

KNeighbors Regressor, SVM Regressor, and Decision Tree Regressor are all non-linear models, and all had less optimal performances than Ridge and Lasso, which are linear models.